# E. Quantization for ResNet using the DeepFake Dataset

## Helper Functions and Classes

In [ ]:
from google.colab import drive
import os
import glob
import random
import numpy as np

drive.mount('/content/drive')
DATA_ROOT = '/content/drive/My Drive/Deep Fake Dataset'
PROCESSED_ROOT = os.path.join(DATA_ROOT, 'processed_faces')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class DeepFakeHybridModel(nn.Module):
    def __init__(self):
        super(DeepFakeHybridModel, self).__init__()

        # 1. Base: EfficientNet-B3 [cite: 131]
        self.backbone = models.efficientnet_b3(pretrained=True)
        self.backbone.classifier = nn.Identity()

        # 2. Custom Layers (The "Head") [cite: 132-135]
        # EfficientNet-B3 output is 1536
        self.dense_512 = nn.Sequential(
            nn.Linear(1536, 512),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.dense_128 = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        # Final Softmax (2 classes)
        self.classifier = nn.Linear(128, 2)

    def forward(self, x):
        x = self.backbone(x)         # 1536
        feat_512 = self.dense_512(x) # 512 (For Ensemble)
        x = self.dense_128(feat_512)
        out = self.classifier(x)     # Classification (For Training)
        return out, feat_512


class FakeQuantWrapper(nn.Module):
    def __init__(self, module: nn.Module, weight_bits:int=8, act_bits:int=8, eps:float=1e-8):
        super().__init__()
        self.module = module
        self.weight_bits = int(weight_bits)
        self.act_bits = int(act_bits)
        self.eps = float(eps)

        # Buffer for cached quantized weight (float tensor) and scale
        self.register_buffer('_w_q_cached', torch.empty(0))
        self.register_buffer('_scale_cached', torch.empty(0))
        self._cache_initialized = False

    def update_cached_weight(self, device=None):
        """Call this after optimizer.step() to refresh the cached quantized weight."""
        w = self.module.weight.data
        if device is not None:
            w = w.to(device)
        # use per-channel quant when weight is multi-dimensional
        if w.ndim >= 2:
            w_q, scale = quantize_tensor_symmetric_perchannel(w, self.weight_bits, eps=self.eps, channel_dim=0)
        else:
            w_q, scale = quantize_tensor_symmetric_perchannel(w, self.weight_bits, eps=self.eps, channel_dim=0)
        # store as buffer on module's device
        self._w_q_cached = w_q.detach().to(self.module.weight.device)
        self._scale_cached = scale.detach().to(self.module.weight.device)
        self._cache_initialized = True

    def forward(self, x):
        # ensure cache exists (lazy init)
        if not self._cache_initialized or self._w_q_cached.numel() == 0:
            w = self.module.weight.data
            if w.ndim >= 2:
                w_q, scale = quantize_tensor_symmetric_perchannel(w, self.weight_bits, eps=self.eps, channel_dim=0)
            else:
                w_q, scale = quantize_tensor_symmetric_perchannel(w, self.weight_bits, eps=self.eps, channel_dim=0)
            self._w_q_cached = w_q.detach().to(self.module.weight.device)
            self._scale_cached = scale.detach().to(self.module.weight.device)
            self._cache_initialized = True

        w_q_cached = self._w_q_cached

        # STE wrapper: (w_q_cached - w).detach() + w
        if isinstance(self.module, nn.Conv2d):
            bias = self.module.bias
            w = self.module.weight
            w_q_for_backward = (w_q_cached - w).detach() + w
            out = F.conv2d(
                x,
                w_q_for_backward,
                bias=bias,
                stride=self.module.stride,
                padding=self.module.padding,
                dilation=self.module.dilation,
                groups=self.module.groups
            )
        elif isinstance(self.module, nn.Linear):
            bias = self.module.bias
            w = self.module.weight
            w_q_for_backward = (w_q_cached - w).detach() + w
            out = F.linear(x, w_q_for_backward, bias)
        else:
            out = self.module(x)

        # Activation quantization (unchanged logic, but clamp scale)
        if self.act_bits < 32:
            qmax = 2 ** (self.act_bits - 1) - 1
            max_abs = out.abs().amax()
            eps_t = torch.tensor(self.eps, device=out.device, dtype=out.dtype)
            scale = (max_abs / qmax).clamp(min=eps_t)
            q = torch.round(out / scale).clamp(-qmax, qmax)
            out_q = q * scale
            out = (out_q - out).detach() + out

        return out


In [ ]:
!cp "/content/drive/MyDrive/Deep Fake Dataset/all.zip" /content/


In [ ]:
!ls -lh /content/all.zip



-rw------- 1 root root 335M Dec  8 18:27 /content/all.zip


In [ ]:
!unzip /content/all.zip -d /content/all_unzipped

Streaming output truncated to the last 5000 lines.
  inflating: /content/all_unzipped/all/032000_3.jpg  
  inflating: /content/all_unzipped/all/032000_1.jpg  
  inflating: /content/all_unzipped/all/032000_2.jpg  
  inflating: /content/all_unzipped/all/019547_8.jpg  
  inflating: /content/all_unzipped/all/020541_8.jpg  
  inflating: /content/all_unzipped/all/020541_6.jpg  
  inflating: /content/all_unzipped/all/008828_7.jpg  
  inflating: /content/all_unzipped/all/020541_1.jpg  
  inflating: /content/all_unzipped/all/008023_8.jpg  
  inflating: /content/all_unzipped/all/020541_2.jpg  
  inflating: /content/all_unzipped/all/020541_7.jpg  
  inflating: /content/all_unzipped/all/008023_5.jpg  
  inflating: /content/all_unzipped/all/008023_9.jpg  
  inflating: /content/all_unzipped/all/008023_7.jpg  
  inflating: /content/all_unzipped/all/020541_5.jpg  
  inflating: /content/all_unzipped/all/020541_3.jpg  
  inflating: /content/all_unzipped/all/027556_1.jpg  
  inflating: /content/all_unzip

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from tqdm.auto import tqdm
import os
import torch.nn.functional as F
import glob
import numpy as np
import os
from PIL import Image
from tqdm.auto import tqdm
import torch
import torchvision.transforms as T
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

train_tf = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dir = "/content/processed_faces"
train_dataset = ImageFolder(train_dir, transform=train_tf)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 2. Setup Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import glob
import numpy as np
import os
from PIL import Image
from tqdm.auto import tqdm
import torch
import torchvision.transforms as T
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report


calib_loader = train_loader
loss_fn = nn.CrossEntropyLoss()


def extract_features_from_disk(split_name, model):
    """
    Extracts 512-dim features from the Dense layer of the trained model.
    Groups frames by video to create a single feature vector per video.
    """
    model.eval()
    dataset_path = os.path.join("/content/processed_faces", split_name)

    # Dictionary to group features by video: { 'video_name': [feat1, feat2...] }
    video_features = {}
    video_labels = {}

    # Get all jpgs recursively
    all_images = glob.glob(os.path.join(dataset_path, '*/*.jpg'))

    print(f"Extracting features from {len(all_images)} frames in {split_name} set...")

    # Must match the transforms used during training
    transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    with torch.no_grad():
        for img_path in tqdm(all_images):
            # 1. Parse Metadata from Filename
            # Path structure: .../train/real/video_name_frame0.jpg
            filename = os.path.basename(img_path)
            # Remove '_frameX.jpg' to recover the unique video ID
            vid_name = filename.rsplit('_', 1)[0]

            # Get label from parent folder name ('real' or 'fake')
            parent_folder = os.path.basename(os.path.dirname(img_path))
            label = 0 if parent_folder == 'real' else 1

            # 2. Load and Preprocess Image
            # .convert('RGB') ensures we don't crash on Alpha channels
            img = Image.open(img_path).convert('RGB')
            img_t = transform(img).unsqueeze(0).to(device)

            # 3. Hybrid Forward Pass
            # We ignore the first return (classification logits)
            # We capture the second return (the 512-dim embedding) [cite: 135]
            _, feat_512 = model(img_t)

            # Flatten to 1D array (512,)
            feat_np = feat_512.cpu().numpy().flatten()

            # 4. Group by Video
            if vid_name not in video_features:
                video_features[vid_name] = []
                video_labels[vid_name] = label

            video_features[vid_name].append(feat_np)

    # 5. Aggregate: Average all frame features to get one vector per video
    final_X = []
    final_y = []
    for vid, feats in video_features.items():
        # Mean of (N_frames, 512) -> (512,)
        avg_feat = np.mean(feats, axis=0)
        final_X.append(avg_feat)
        final_y.append(video_labels[vid])

    return np.array(final_X), np.array(final_y)

# small utility to iterate model weight parameters (exclude biases)
def named_weight_params(model):
    for name, p in model.named_parameters():
        if p.requires_grad and p.ndim >= 1 and 'bias' not in name:
            yield name, p

def hutchinson_diag(model, loss_fn, dataloader, device='cuda', num_vectors=8, max_batches=2):
    model.to(device)
    model.eval()
    param_items = list(named_weight_params(model))
    param_list = [p for _, p in param_items]
    param_names = [n for n, _ in param_items]
    diag_accum = [torch.zeros_like(p, device=device) for p in param_list]
    total_draws = 0

    for batch_idx, batch in enumerate(dataloader):
        if max_batches is not None and batch_idx >= max_batches:
            break
        model.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Handle Tuple output (Hybrid Model)
        outputs = model(inputs)
        if isinstance(outputs, tuple):
            logits = outputs[0]
        else:
            logits = outputs

        loss = loss_fn(logits, targets)

        # 1. Create the graph once
        grads = torch.autograd.grad(loss, param_list, create_graph=True)

        for _ in range(num_vectors):
            vs = [ (torch.randint_like(p, low=0, high=2, device=device).float() * 2 - 1) for p in param_list ]
            inner = 0.0
            for g, v in zip(grads, vs):
                inner = inner + (g * v).sum()

            # 2. FIX: retain_graph=True so we don't delete 'grads' graph
            # after the first vector.
            Hv = torch.autograd.grad(inner, param_list, retain_graph=True)

            for i, (v, hvi) in enumerate(zip(vs, Hv)):
                diag_accum[i] += (v * hvi).detach()
            total_draws += 1

        # Cleanup
        for g in grads:
            del g

    if total_draws == 0:
        raise RuntimeError("No batches processed in Hutchinson estimator.")
    diag_est = {name: (d / total_draws).cpu() for name, d in zip(param_names, diag_accum)}
    return diag_est

def empirical_fisher_diag(model, loss_fn, dataloader, device='cuda', max_batches=5):
    # cheap fallback: use average squared gradients as proxy for Hessian diagonal
    model.to(device)
    model.eval()
    param_items = list(named_weight_params(model))
    param_list = [p for _, p in param_items]
    param_names = [n for n, _ in param_items]
    accum = [torch.zeros_like(p, device='cpu') for p in param_list]
    count = 0

    for batch_idx, batch in enumerate(dataloader):
        if max_batches is not None and batch_idx >= max_batches:
            break
        model.zero_grad()
        inputs, targets = batch
        inputs = inputs.to(device); targets = targets.to(device)

        # --- FIX: Unpack tuple output ---
        outputs = model(inputs)
        if isinstance(outputs, tuple):
            logits = outputs[0]
        else:
            logits = outputs
        # --------------------------------

        loss = loss_fn(logits, targets)
        grads = torch.autograd.grad(loss, param_list, retain_graph=False)
        for i, g in enumerate(grads):
            accum[i] += (g.detach().cpu() ** 2)
        count += 1

    if count == 0:
        raise RuntimeError("No batches processed for empirical fisher.")
    diag_est = {}
    for (name, p), a in zip(param_items, accum):
        avg_sq = (a / count)
        if avg_sq.shape != p.shape:
            avg_sq = avg_sq.expand_as(p).cpu()
        diag_est[name] = (avg_sq + 1e-12).cpu()
    return diag_est

# (get_diag_estimates remains the same, but calls the fixed functions above)
def get_diag_estimates(model, loss_fn, calib_loader, device='cuda', try_hutchinson=True,
                       hutchinson_vectors=8, hutchinson_batches=2, fisher_batches=5):
    try:
        if try_hutchinson:
            print("Running Hutchinson Hessian diag estimator (may be slow)...")
            return hutchinson_diag(model, loss_fn, calib_loader, device=device,
                                   num_vectors=hutchinson_vectors, max_batches=hutchinson_batches)
    except Exception as e:
        print("Hutchinson failed:", repr(e))
        print("Falling back to empirical Fisher.")
    return empirical_fisher_diag(model, loss_fn, calib_loader, device=device, max_batches=fisher_batches)

def quantize_tensor_symmetric_perchannel(w: torch.Tensor, nbits:int, eps:float = 1e-8, channel_dim:int = 0):
    """
    Symmetric uniform quantization with per-channel support.
    Returns (w_q, scale) where w_q is the dequantized float tensor and
    scale is broadcastable to w (scalar for per-tensor, shape (C,1,1..) for per-channel).
    """
    if nbits < 1:
        raise ValueError("nbits must be >= 1")
    qmax = 2 ** (nbits - 1) - 1
    with torch.no_grad():
        if w.ndim >= 2:
            # per-channel over channel_dim
            reduce_dims = [i for i in range(w.ndim) if i != channel_dim]
            max_abs = w.abs().amax(dim=reduce_dims, keepdim=True)  # shape like (C,1,1,..)
            eps_t = torch.tensor(eps, device=w.device, dtype=w.dtype)
            scale = (max_abs / qmax).clamp(min=eps_t)
            q = torch.round(w / scale).clamp(-qmax, qmax)
            w_q = q * scale
        else:
            # fallback per-tensor
            max_abs = w.abs().amax()
            eps_t = torch.tensor(eps, device=w.device, dtype=w.dtype)
            scale = (max_abs / qmax).clamp(min=eps_t)
            q = torch.round(w / scale).clamp(-qmax, qmax)
            w_q = q * scale

        # final safety: if any non-finite, fallback to original weights
        if not torch.isfinite(w_q).all() or not torch.isfinite(scale).all():
            return w.detach().clone(), scale
    return w_q, scale

def estimated_delta_loss_from_quant(w: torch.Tensor, diag_h: torch.Tensor, w_q: torch.Tensor):
    delta = (w_q - w).detach().cpu()
    h = diag_h.cpu()
    if h.shape != delta.shape:
        try:
            h = h.expand_as(delta)
        except Exception:
            h = torch.ones_like(delta) * float(h.mean().item())
    return 0.5 * (h * (delta ** 2)).sum().item()

def compute_costs(model, diag_h_dict, base_bits=4, max_bits=8):
    costs = {}
    for name, p in named_weight_params(model):
        w = p.data
        diag = diag_h_dict.get(name, torch.ones_like(w.cpu()) * 1e-6)
        costs[name] = {}
        for b in range(base_bits, max_bits+1):
            w_q, _ = quantize_tensor_symmetric_perchannel(w, b)
            costs[name][b] = estimated_delta_loss_from_quant(w, diag, w_q)
    return costs

def allocate_bits_greedy(costs, base_bits=4, max_bits=8, bit_budget=None):
    """
    Greedy bit allocator.
    - If bit_budget is None: choose per-layer best bits by minimizing cost.
    - If bit_budget provided: allocate additional bits one at a time to the layer
      whose single-bit increment gives the largest immediate reduction in cost.
      Unlike the previous implementation, this WILL allocate bits even when the
      immediate gain is zero or negative, until the total budget is exhausted
      (or all layers are at max_bits).
    """
    names = list(costs.keys())
    assignment = {name: base_bits for name in names}

    # If no budget constraint, pick best per-layer bit that minimizes cost
    if bit_budget is None:
        for name in names:
            assignment[name] = min(range(base_bits, max_bits + 1), key=lambda b: costs[name][b])
        return assignment

    total_layers = len(names)
    # Interpret bit_budget as average bits when <= max_bits, else absolute total bits
    if bit_budget <= max_bits:
        total_bits_allowed = int(bit_budget * total_layers)
    else:
        total_bits_allowed = int(bit_budget)

    total_bits = sum(assignment.values())

    # If budget already satisfied or exceeded, return current assignment
    if total_bits >= total_bits_allowed:
        return assignment

    # Greedily allocate remaining bits; allow non-positive gains so we always exhaust budget
    while total_bits < total_bits_allowed:
        best_gain = -float('inf')   # allow selecting negative or zero gains
        best_name = None
        for name in names:
            cur_b = assignment[name]
            if cur_b >= max_bits:
                continue
            # Immediate marginal gain = current_cost - cost_if_incremented
            gain = costs[name][cur_b] - costs[name][cur_b + 1]
            if gain > best_gain:
                best_gain = gain
                best_name = name

        # If no eligible layer to increase (all at max) break early
        if best_name is None:
            break

        # Assign the chosen bit and continue
        assignment[best_name] += 1
        total_bits += 1

    return assignment

def update_all_fakequant_caches(model, device=None):
    """Walk modules and call update_cached_weight() for each FakeQuantWrapper."""
    for m in model.modules():
        if isinstance(m, FakeQuantWrapper):
            m.update_cached_weight(device=device)


def wrap_model_with_fakequant(model, bits_assignment, act_bits=8):
    """
    bits_assignment: dict mapping named parameter names (weight param names) -> bitwidth (int)
    We will find parent modules that own those parameter names and wrap Conv2d/Linear modules.
    """
    model = model  # modify in place
    # Build map param_name -> parent module and attribute name
    name_to_parent = {}
    for p_name, p in model.named_parameters(recurse=True):
        if p_name in bits_assignment and p_name.endswith('weight'):
            parts = p_name.split('.')
            parent = model
            for part in parts[:-1]:
                parent = getattr(parent, part)
            # parent is module owning the parameter
            name_to_parent[p_name] = (parent, parts[:-1])  # parent module and path
    # To avoid replacing same module multiple times, track modules wrapped
    wrapped = set()
    for p_name, (parent_module, path) in name_to_parent.items():
        # path last element is module attribute name in parent's parent
        if len(path) == 0:
            continue
        # find grandparent to set attribute
        grandparent = model
        for part in path[:-1]:
            grandparent = getattr(grandparent, part)
        attr = path[-1]
        orig_mod = getattr(grandparent, attr)
        if id(orig_mod) in wrapped:
            continue
        # only wrap Conv2d and Linear (others we skip)
        if isinstance(orig_mod, (nn.Conv2d, nn.Linear)):
            wbits = bits_assignment.get(p_name, 8)
            wrapped_mod = FakeQuantWrapper(orig_mod, weight_bits=wbits, act_bits=act_bits)
            setattr(grandparent, attr, wrapped_mod)
            wrapped.add(id(wrapped_mod))
    return model

def qat_train(
    model,
    train_loader,
    val_loader,
    device,
    epochs: int = 6,
    lr: float = 1e-4,
    weight_decay: float = 0.0,
    max_grad_norm: float = 1.0,
    scale_eps: float = 1e-8,
    use_amp: bool = False,
    anomaly_detection: bool = False,
    recompute_bn: bool = True,
    recompute_bn_batches: int = 200,
    log_every: int = 50,
):
    """
    Robust QAT training loop with:
      - quantizer scale clamping (patches quantize_tensor_symmetric if present)
      - gradient clipping and batch skipping on NaN/Inf grads or loss
      - optional AMP and anomaly detection
      - automatic cache updates for FakeQuantWrapper (expects update_all_fakequant_caches())
      - optional BN running-stat recomputation after training

    Usage: drop in place of your old qat_train. Returns (model, val_acc_percent).

    Args:
      model: nn.Module (already constructed; will be moved to device inside).
      train_loader, val_loader: torch DataLoader objects.
      device: 'cuda' or 'cpu'
      epochs: number of QAT epochs
      lr: learning rate (default 1e-4)
      weight_decay: (default 0.0)
      max_grad_norm: clip grad norm
      scale_eps: clamp for quantization scales (prevents divide-by-tiny)
      use_amp: whether to use torch.cuda.amp (False by default)
      anomaly_detection: enable torch.autograd.set_detect_anomaly (slow)
      recompute_bn: whether to recompute BN running stats after training
      recompute_bn_batches: number of batches to run for BN stat recompute
      log_every: print every N iterations
    """

    # ----------------------- Safety: patch quantizer (if present) -----------------------
    # Replace quantize_tensor_symmetric with a safe clamped version (per-tensor).
    # This is a monkeypatch so it only affects this session; it's conservative.
    try:
        _globals = globals()
        if 'quantize_tensor_symmetric' in _globals:
            # keep original for reference
            _globals['_original_quantize_tensor_symmetric'] = _globals['quantize_tensor_symmetric']
        def _quantize_tensor_symmetric_clamped(w: torch.Tensor, nbits:int, eps:float = scale_eps):
            # robust symmetric per-tensor quantization with scale clamp
            if nbits < 1:
                raise ValueError("nbits must be >= 1")
            qmax = 2 ** (nbits - 1) - 1
            with torch.no_grad():
                max_abs = w.abs().max()
                eps_t = torch.tensor(eps, device=w.device, dtype=w.dtype)
                scale = (max_abs / qmax).clamp(min=eps_t)
                q = torch.round(w / scale).clamp(-qmax, qmax)
                w_q = q * scale
                # if non-finite, fallback to original weights (safe)
                if not torch.isfinite(w_q).all() or not torch.isfinite(scale).all():
                    return w.detach().clone(), scale
            return w_q, scale
        # set/override in global namespace
        _globals['quantize_tensor_symmetric'] = _quantize_tensor_symmetric_clamped
        # Also patch a common "no_grad" variant name if present
        if 'quantize_tensor_symmetric_no_grad' in _globals:
            _globals['_original_quantize_tensor_symmetric_no_grad'] = _globals['quantize_tensor_symmetric_no_grad']
            _globals['quantize_tensor_symmetric_no_grad'] = lambda w, nbits: _quantize_tensor_symmetric_clamped(w, nbits, eps=scale_eps)
    except Exception as e:
        print("Warning: failed to monkeypatch quantizer:", e)
    # ------------------------------------------------------------------------------------

    model.to(device)
    # Create optimizer AFTER any wrapping is done by caller (this function assumes model is already wrapped if desired).
    opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay, eps=1e-8)

    loss_local = loss_fn  # expects this to be defined in surrounding scope

    # Warm-up: initialize caches for fake-quant (if applicable)
    try:
        update_all_fakequant_caches(model, device=device)
    except Exception:
        # if update helper not defined or wrappers not present, ignore
        pass

    scaler = torch.cuda.amp.GradScaler() if use_amp else None

    if anomaly_detection:
        torch.autograd.set_detect_anomaly(True)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        n = 0

        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader),
                            desc=f"Epoch {epoch+1}/{epochs}")

        for i, (xb, yb) in progress_bar:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()

            try:
                if use_amp:
                    with torch.cuda.amp.autocast():
                        out_tuple = model(xb)
                        # support both single-output and tuple (logits, features)
                        logits = out_tuple[0] if isinstance(out_tuple, (tuple, list)) else out_tuple
                        loss = loss_local(logits, yb)
                else:
                    out_tuple = model(xb)
                    logits = out_tuple[0] if isinstance(out_tuple, (tuple, list)) else out_tuple
                    loss = loss_local(logits, yb)
            except Exception as e:
                print(f"Forward error on batch {i}: {e}. Skipping batch.")
                continue

            # Check loss finite
            if not torch.isfinite(loss):
                print(f"Warning: non-finite loss at epoch {epoch+1} batch {i}. Skipping batch.")
                continue

            # Backward (with optional AMP)
            if use_amp:
                scaler.scale(loss).backward()
            else:
                loss.backward()

            # Check grads for non-finite values; if found, skip batch (and zero grads)
            bad_grad = False
            for name, p in model.named_parameters():
                if p.grad is None:
                    continue
                if not torch.isfinite(p.grad).all():
                    print(f"Non-finite gradient detected in {name} at epoch {epoch+1} batch {i}. Skipping batch.")
                    bad_grad = True
                    break
            if bad_grad:
                opt.zero_grad()
                continue

            # Gradient clipping
            if use_amp:
                # unscale then clip
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                scaler.step(opt)
                scaler.update()
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                opt.step()

            # Refresh cached quantized weights if your wrappers need it
            try:
                update_all_fakequant_caches(model, device=device)
            except Exception:
                pass

            running_loss += float(loss.item())
            n += 1

            # occasional logging
            if (i + 1) % log_every == 0:
                avg_loss = running_loss / max(1, n)
                progress_bar.set_postfix({'avg_loss': f"{avg_loss:.4f}"})

        # end epoch
        if n > 0:
            print(f"Epoch {epoch+1} training loss {(running_loss/n):.4f}")
        else:
            print(f"Epoch {epoch+1} no successful batches processed (all skipped due to NaNs?)")

    # turn off anomaly detection if enabled earlier
    if anomaly_detection:
        torch.autograd.set_detect_anomaly(False)

    # ------------------ Recompute BN running stats (optional) ------------------
    if recompute_bn:
        def recompute_bn_stats(model_, loader_, device_, num_batches_=recompute_bn_batches):
            model_.train()
            with torch.no_grad():
                it = iter(loader_)
                for _i in range(num_batches_):
                    try:
                        xb_, _ = next(it)
                    except StopIteration:
                        it = iter(loader_)
                        xb_, _ = next(it)
                    xb_ = xb_.to(device_)
                    _ = model_(xb_)
        try:
            print("Recomputing BN running statistics...")
            recompute_bn_stats(model, train_loader, device, recompute_bn_batches)
        except Exception as e:
            print("Warning: recompute BN failed:", e)
    # ---------------------------------------------------------------------------

    # ------------------ After QAT: extract features and evaluate ensemble ------------------
    try:
        X_train, y_train = extract_features_from_disk('train', model)
        X_val, y_val = extract_features_from_disk('val', model)
    except Exception as e:
        print("Error during feature extraction/eval:", e)
        return model, None

    clfs = {
      'SVM': SVC(probability=True, kernel='rbf'),
      'DT': DecisionTreeClassifier(),
      'KNN': KNeighborsClassifier(n_neighbors=5),
      'NB': GaussianNB()
    }

    preds_stack = []
    print("\n--- Ensemble Classifier Performance ---")
    for name, clf in clfs.items():
        clf.fit(X_train, y_train)
        p = clf.predict(X_val)
        preds_stack.append(p)

    preds_stack = np.array(preds_stack)
    votes = preds_stack.sum(axis=0)
    ensemble_pred = (votes >= 2).astype(int)
    acc = accuracy_score(y_val, ensemble_pred) * 100.0
    print(f"Ensemble accuracy: {acc:.2f}%")

    return model, acc

def is_weight_param(name, param):
    return param.requires_grad and param.ndim >= 1 and 'bias' not in name

# 2) build list of param names that we WANT to quantize (exclude head prefixes)
def quantizable_param_names(model, exclude_prefixes=('dense_128', 'classifier')):
    names = []
    for n,p in model.named_parameters(recurse=True):
        if not is_weight_param(n,p):
            continue
        # exclude any param whose name starts with a prefix in exclude_prefixes
        if any(n.startswith(pref) for pref in exclude_prefixes):
            continue
        names.append(n)
    return names

# 3) compute_costs but only for params in include_names (keeps original behavior otherwise)
def compute_costs_filtered(model, diag_h_dict, include_names=None, base_bits=4, max_bits=8):
    costs = {}
    for name, p in named_weight_params(model):
        if include_names is not None and name not in include_names:
            # skip computing cost for excluded params
            continue
        w = p.data
        diag = diag_h_dict.get(name, torch.ones_like(w.cpu()) * 1e-6)
        costs[name] = {}
        for b in range(base_bits, max_bits+1):
            w_q, _ = quantize_tensor_symmetric_perchannel(w, b)
            costs[name][b] = estimated_delta_loss_from_quant(w, diag, w_q)
    return costs


# --- Cell: Save quantized weights efficiently given a bit assignment ---
import torch, os, math
from collections import OrderedDict

def _quantize_tensor_symmetric_perchannel_to_int(t: torch.Tensor, bits: int, eps: float = 1e-8, channel_dim:int = 0):
    """
    Quantize tensor t (fp32) symmetrically with per-channel scales.
    Returns (q_int_tensor, scale_tensor, int_dtype_str)
      - q_int_tensor: integer tensor (torch.int8/int16/int32) containing quantized integers
      - scale_tensor: float32 tensor broadcastable to t.shape that gives dequant: w_fp = q_int.float() * scale
      - dtype_str: 'int8'/'int16'/'int32' used to store q_int_tensor
    """
    if bits < 1:
        raise ValueError("bits must be >=1")
    qmax = 2 ** (bits - 1) - 1
    # per-channel scale
    if t.ndim >= 2:
        reduce_dims = [i for i in range(t.ndim) if i != channel_dim]
        max_abs = t.abs().amax(dim=reduce_dims, keepdim=True)
        scale = (max_abs / qmax).clamp(min=eps)
    else:
        max_abs = t.abs().amax()
        scale_val = (max_abs / qmax).clamp(min=eps)
        scale = scale_val

    # compute integer representation
    q = torch.round(t / scale).clamp(-qmax, qmax)

    # choose storage dtype
    if bits <= 8:
        q_int = q.to(torch.int8)
        dtype_str = 'int8'
    elif bits <= 16:
        q_int = q.to(torch.int16)
        dtype_str = 'int16'
    else:
        q_int = q.to(torch.int32)
        dtype_str = 'int32'

    # store scale as float32 (broadcastable)
    scale = scale.to(torch.float32)

    return q_int, scale, dtype_str

def save_quantized_checkpoint(fp32_ckpt_path: str,
                              bits_assignment: dict,
                              out_path: str,
                              exclude_prefixes: tuple = ('dense_128', 'classifier'),
                              eps: float = 1e-8):
    """
    Args:
      fp32_ckpt_path: path to saved FP32 state_dict (torch .pth/.pt)
      bits_assignment: dict {param_name (str) : bits (int)} for params to quantize
                       (should include only weight names you want to quantize).
      out_path: output .pth path to save quantized representation (torch.save(dict))
      exclude_prefixes: tuple of prefixes to skip (e.g., head layers kept FP)
      eps: minimum scale to avoid divide-by-zero
    Produces a single file at out_path with structure:
      {
        'quant': { param_name: { 'q': Tensor(int*), 'scale': Tensor(float32), 'bits': int, 'dtype':str, 'shape': tuple } },
        'fp32_fallback': { param_name: Tensor(fp32) }  # for excluded or non-quantized params
        'meta': {...}
      }
    """
    # 1) load fp32 checkpoint
    print("Loading FP32 checkpoint:", fp32_ckpt_path)
    ckpt = torch.load(fp32_ckpt_path, map_location='cpu')
    # ckpt might be a state_dict or dict with 'model_state' etc. Detect:
    if isinstance(ckpt, dict) and any(k.endswith('weight') or '.' in k for k in ckpt.keys()):
        state_dict = ckpt
    elif isinstance(ckpt, dict) and 'state_dict' in ckpt:
        state_dict = ckpt['state_dict']
    else:
        state_dict = ckpt  # assume it's the raw state_dict

    quant_store = {}
    fp32_fallback = {}
    total_fp32_bytes = 0
    total_quant_bytes = 0

    for name, tensor in state_dict.items():
        # only quantize weight parameters (skip biases and 0/1-D if desired)
        is_bias = name.lower().endswith('bias')
        if any(name.startswith(pref) for pref in exclude_prefixes) or is_bias:
            # keep in fp32 fallback (do not quantize)
            fp32_fallback[name] = tensor.clone().to(torch.float32)
            total_fp32_bytes += tensor.numel() * 4
            continue

        # If this param wasn't assigned bits, skip (store fp32), else quantize it
        bits = bits_assignment.get(name, None)
        if bits is None:
            fp32_fallback[name] = tensor.clone().to(torch.float32)
            total_fp32_bytes += tensor.numel() * 4
            continue

        # perform per-channel symmetric quantization to integer
        w = tensor.clone().to(torch.float32)
        q_int, scale, dtype_str = _quantize_tensor_symmetric_perchannel_to_int(w, bits, eps=eps, channel_dim=0)

        quant_store[name] = {
            'q': q_int,               # int tensor
            'scale': scale,           # float32 tensor
            'bits': int(bits),
            'dtype': dtype_str,
            'shape': tuple(w.shape)
        }

        # accumulate sizes
        total_quant_bytes += q_int.numel() * (1 if dtype_str=='int8' else (2 if dtype_str=='int16' else 4))
        total_quant_bytes += scale.numel() * 4  # scales stored as float32

    # meta
    meta = {
        'num_quantized': len(quant_store),
        'num_fallback_fp32': len(fp32_fallback),
        'total_fp32_bytes_preexisting': total_fp32_bytes,
        'total_quant_bytes_after': total_quant_bytes
    }

    out = {
        'quant': quant_store,
        'fp32_fallback': fp32_fallback,
        'meta': meta
    }

    # final save
    torch.save(out, out_path)
    print(f"Saved quantized checkpoint to {out_path}")
    print("Meta:", meta)
    approx_ratio = (total_fp32_bytes / total_quant_bytes) if total_quant_bytes>0 else float('inf')
    print(f"Approx compression ratio (FP32 bytes / quant bytes): {approx_ratio:.2f}x (higher is better)")

    return out_path

def dequantize_state_dict_from_disk(quant_path: str, device='cpu'):
    """
    Loads the quant file produced above and reconstructs a full FP32 state_dict ready to load into your model.
    This will dequantize stored integer tensors back to float32 using stored scales and put fp32_fallback back.
    """
    data = torch.load(quant_path, map_location='cpu')
    out_state = OrderedDict()

    # first add fp32 fallback params
    for name, t in data.get('fp32_fallback', {}).items():
        out_state[name] = t.to(torch.float32).clone()

    # then dequantize quantized params
    for name, info in data.get('quant', {}).items():
        q = info['q']           # integer tensor on CPU
        scale = info['scale']   # float tensor broadcastable
        # convert q to float then dequantize
        qf = q.to(torch.float32)
        w_fp = qf * scale
        out_state[name] = w_fp.to(torch.float32).clone()

    return out_state

# ------------------------------
# Example usage (edit paths & variables)
# ------------------------------
# fp32_ckpt_path = "/content/efficientnet_deepfake_v1_30epoch.pth"
# bits_assignment = bits_assignment_partial   # the mapping your allocator produced
# out_path = "/content/efficientnet_quantized_mixedbits.pth"
# save_quantized_checkpoint(fp32_ckpt_path, bits_assignment, out_path, exclude_prefixes=('dense_128','classifier'), eps=1e-8)
#
# # To load later and reconstruct FP32 state_dict:
# state_dict_fp32 = dequantize_state_dict_from_disk(out_path, device='cpu')
# # then you can do model.load_state_dict(state_dict_fp32)



##Start Training

In [ ]:
CHECKPOINT_PATH = os.path.join(DATA_ROOT, 'trained_cnn', 'efficientnet_deepfake_v1_30epoch.pth')

exclude_prefixes = ('dense_128', 'classifier')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
try_hutchinson = True            # try true Hessian diag first
hutch_vectors = 16                # 6..12 on Colab, 12..16 on V100
hutch_batches = 2                # small number of batches for speed
fisher_batches = 6               # fallback batch count
base_bits = 4                    # minimum bits considered
max_bits = 16                     # maximum bits to consider (keep 8)
activation_bits = 8              # activation fake-quant bits (common across layers)
qat_epochs = 5                   # fine-tune epochs (2-8)
qat_lr = 2e-5                    # learning rate for QAT

bit_budgets = [8]

val_accs = {}
for bit_budget in bit_budgets:
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  efficientnet_model = DeepFakeHybridModel().to(device)

# Logic to load or initialize blank
  if os.path.exists(CHECKPOINT_PATH):
      print(f"Loading weights from {CHECKPOINT_PATH}...")
      efficientnet_model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=device))
      print("Model loaded successfully.")
  diag_h = get_diag_estimates(efficientnet_model, loss_fn, calib_loader, device=device,
                            try_hutchinson=try_hutchinson,
                            hutchinson_vectors=hutch_vectors,
                            hutchinson_batches=hutch_batches,
                            fisher_batches=fisher_batches)

  # 2) build a list of param names we want to quantize (backbone + dense_512)
  include_names = quantizable_param_names(efficientnet_model, exclude_prefixes=exclude_prefixes)
  print("Will quantize these params (sample):", include_names[:10])

  # 3) compute costs only for the included params
  costs = compute_costs_filtered(efficientnet_model, diag_h, include_names=include_names,
                                base_bits=base_bits, max_bits=max_bits)

  # 4) allocate bits greedily for only these params (this returns dict for included names)
  bits_assignment_partial = allocate_bits_greedy(costs, base_bits=base_bits, max_bits=max_bits, bit_budget=bit_budget)

  # 5) If you'd like, explicitly set excluded params to full precision by not including them.
  # bits_assignment_partial now contains only quantized params; wrap only those.
  print("Assigned bits (sample):")
  for i,(k,v) in enumerate(bits_assignment_partial.items()):
      if i >= 25: break
      print(f"  {k} -> {v}")

  # 6) Wrap model: wrap_model_with_fakequant will only wrap modules that match names in bits_assignment
  qat_model = efficientnet_model.train()
  qat_model = wrap_model_with_fakequant(qat_model, bits_assignment_partial, act_bits=activation_bits)
  qat_model.to(device)

  # 7) Create optimizer AFTER wrapping so it includes both FP params (head) and wrapped params
  opt = torch.optim.AdamW(qat_model.parameters(), lr=qat_lr, weight_decay=1e-4)

  # 4) Run QAT fine-tuning
  qat_model, final_val_acc = qat_train(qat_model, train_loader, calib_loader, device, epochs=qat_epochs, lr=qat_lr)
  torch.save(qat_model.state_dict(), f"qat_model_bits_{bit_budget}.pth")
  val_accs[bit_budget] = final_val_acc

  print(val_accs)


fp32_ckpt_path = "/content/qat_model_bits_8.pth"
bits_assignment = bits_assignment_partial
out_path = os.path.join(DATA_ROOT, 'trained_cnn', 'quantized8_model_10epoch.pth')
save_quantized_checkpoint(fp32_ckpt_path, bits_assignment, out_path, exclude_prefixes=('dense_128','classifier'), eps=1e-8)



/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading weights from /content/drive/My Drive/Deep Fake Dataset/trained_cnn/efficientnet_deepfake_v1_30epoch.pth...
Model loaded successfully.
Running Hutchinson Hessian diag estimator (may be slow)...
Will quantize these params (sample): ['backbone.features.0.0.weight', 'backbone.features.0.1.weight', 'backbone.features.1.0.block.0.0.weight', 'backbone.features.1.0.block.0.1.weight', 'backbone.features.1.0.block.1.fc1.weight', 'backbone.features.1.0.block.1.fc2.weight', 'backbone.features.1.0.block.2.0.weight', 'backbone.features.1.0.block.2.1.weight', 'backbone.features.1.1.block.0.0.weight', 'backbone.features.1.1.block.0.1.weight']
Assigned bits (sample):
  backbone.features.0.0.weight -> 4
  backbone.features.0.1.weight -> 4
  backbone.features.1.0.block.0.0.weight -> 4
  backbone.features.1.0.block.0.1.weight -> 8
  backbone.features.1.0.block.1.fc1.weight -> 16
  backbone.features.1.0.block.1.fc2.weight -> 4
  backbone.features.1.0.block.2.0.weight -> 4
  backbone.features.1.0.bl

Epoch 1/5:   0%|          | 0/625 [00:00<?, ?it/s]

Epoch 1 training loss 5.1307


Epoch 2/5:   0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/16000 [00:00<?, ?it/s]

Extracting features from 4000 frames in val set...


  0%|          | 0/4000 [00:00<?, ?it/s]


--- Ensemble Classifier Performance ---
Ensemble accuracy: 92.50%
{8: 92.5}
Loading FP32 checkpoint: /content/qat_model_bits_8.pth
Saved quantized checkpoint to /content/drive/My Drive/Deep Fake Dataset/trained_cnn/quantized8_model_10epoch.pth
Meta: {'num_quantized': 78, 'num_fallback_fp32': 762, 'total_fp32_bytes_preexisting': 92129408, 'total_quant_bytes_after': 52296}
Approx compression ratio (FP32 bytes / quant bytes): 1761.69x (higher is better)


'/content/drive/My Drive/Deep Fake Dataset/trained_cnn/quantized8_model_10epoch.pth'

In [ ]:
# --- Cell: remap wrappers -> canonical names then quantize & save compact checkpoint ---
import torch, os, math
from collections import OrderedDict, Counter

# EDIT THESE
fp32_ckpt_path = "/content/qat_model_bits_8.pth"   # the checkpoint you currently have (wrapped model)
out_path = "/content/drive/My Drive/Deep Fake Dataset/trained_cnn/quantized_all8_mapped.pth"
exclude_prefixes = ('dense_128', 'classifier')    # keep these in FP32 fallback
default_bits = 8
eps = 1e-8

# load the wrapped model state_dict
raw = torch.load(fp32_ckpt_path, map_location='cpu')
if isinstance(raw, dict) and 'state_dict' in raw:
    sd = raw['state_dict']
else:
    sd = raw  # assume raw is state_dict

print("Loaded state_dict with", len(sd), "entries.")

# Build canonical name map:
# Strategy:
#  - If key ends with '.module.weight' -> canonical = key.replace('.module.weight', '.weight')
#  - Else if key ends with '._w_q_cached' -> canonical = key.replace('._w_q_cached', '.weight')
#  - Else keep as-is.
# We'll prefer actual underlying tensors (module.weight or weight) and skip the cached buffers.
canonical_map = {}   # canonical_name -> (best_key_in_sd, tensor)
conflicts = {}

for k, t in sd.items():
    # compute candidate canonical name
    if k.endswith('.module.weight'):
        can = k[:-len('.module.weight')] + '.weight'
    elif k.endswith('._w_q_cached'):
        can = k[:-len('._w_q_cached')] + '.weight'
    else:
        can = k

    # prefer direct '...weight' or '...module.weight' keys (actual parameter tensors) over caches
    # Score preference: prefer keys that contain '.module.weight' or endwith '.weight' (not cached)
    score = 0
    if k.endswith('.module.weight'):
        score = 3
    elif k.endswith('.weight'):
        score = 2
    elif k.endswith('._w_q_cached'):
        score = 1
    else:
        # other buffer/param, give default score 0
        score = 0

    # choose best available tensor for canonical name
    if can not in canonical_map:
        canonical_map[can] = (k, t.clone().to(torch.float32), score)
    else:
        # compare score and replace if current is better
        _, _, prev_score = canonical_map[can]
        if score > prev_score:
            canonical_map[can] = (k, t.clone().to(torch.float32), score)
        elif score == prev_score and k != canonical_map[can][0]:
            # record conflict (non-fatal)
            conflicts.setdefault(can, []).append(k)

# finalize mapping: strip scores, keep canonical->tensor
canon_to_tensor = {can: info[1] for can, info in canonical_map.items()}

print("Canonical params found:", len(canon_to_tensor))
if conflicts:
    print("Conflicts (canonical name -> alternative keys):", {k: v for k, v in list(conflicts.items())[:10]})
    # you can inspect conflicts if desired

# Now choose which canonical params to quantize:
# bits_assignment (from your allocator) typically uses canonical names like '...weight'
try:
    ba = bits_assignment_partial  # use your in-memory mapping (original names)
except NameError:
    print("bits_assignment_partial not found. Using default_bits for all quantizable params.")
    ba = {}

quant_store = {}
fp32_fallback = {}
total_quant_bytes = 0
total_fallback_bytes = 0

def _quantize_tensor_symmetric_perchannel_to_int(t: torch.Tensor, bits: int, eps: float = 1e-8, channel_dim:int = 0):
    qmax = 2 ** (bits - 1) - 1
    if t.ndim >= 2:
        reduce_dims = [i for i in range(t.ndim) if i != channel_dim]
        max_abs = t.abs().amax(dim=reduce_dims, keepdim=True)
        scale = (max_abs / qmax).clamp(min=eps)
    else:
        max_abs = t.abs().amax()
        scale = (max_abs / qmax).clamp(min=eps)
    q = torch.round(t / scale).clamp(-qmax, qmax)
    if bits <= 8:
        q_int = q.to(torch.int8); dtype_str = 'int8'
    elif bits <= 16:
        q_int = q.to(torch.int16); dtype_str = 'int16'
    else:
        q_int = q.to(torch.int32); dtype_str = 'int32'
    return q_int, scale.to(torch.float32), dtype_str

# iterate canonical params
for can_name, tensor in canon_to_tensor.items():
    # skip non-weight like things that are not trainable weights (conservative heuristic)
    if not can_name.endswith('.weight'):
        # keep these in fallback
        fp32_fallback[can_name] = tensor.clone()
        total_fallback_bytes += tensor.numel() * 4
        continue

    # skip excluded prefixes (your classifier layers)
    if any(can_name.startswith(pref) for pref in exclude_prefixes):
        fp32_fallback[can_name] = tensor.clone()
        total_fallback_bytes += tensor.numel() * 4
        continue

    # decide bits: prefer bits from ba, else default_bits
    bits = ba.get(can_name, default_bits)
    # quantize
    q_int, scale, dtype_str = _quantize_tensor_symmetric_perchannel_to_int(tensor, bits, eps=eps, channel_dim=0)
    quant_store[can_name] = {'q': q_int, 'scale': scale, 'bits': int(bits), 'dtype': dtype_str, 'shape': tuple(tensor.shape)}
    total_quant_bytes += q_int.numel() * (1 if dtype_str=='int8' else (2 if dtype_str=='int16' else 4))
    total_quant_bytes += scale.numel() * 4

print("Prepared quant_store entries:", len(quant_store))
print("Prepared fp32_fallback entries:", len(fp32_fallback))
print("Bytes: quant ints+scales:", total_quant_bytes, "fallback fp32:", total_fallback_bytes)

# Save compact file
out = {'quant': quant_store, 'fp32_fallback': fp32_fallback, 'meta': {'num_quantized': len(quant_store), 'num_fallback_fp32': len(fp32_fallback), 'total_quant_bytes': total_quant_bytes, 'total_fallback_bytes': total_fallback_bytes}}
torch.save(out, out_path)
disk_bytes = os.path.getsize(out_path)
orig_bytes = sum(t.numel()*4 for t in canon_to_tensor.values())
print("Saved to:", out_path)
print("Original (canon) FP32 total bytes (approx):", orig_bytes, " (~{:.2f} MB)".format(orig_bytes/1024**2))
print("Quant stored ints+scales bytes (sum):", total_quant_bytes, " (~{:.2f} KB)".format(total_quant_bytes/1024))
print("Fallback FP32 bytes stored:", total_fallback_bytes, " (~{:.2f} MB)".format(total_fallback_bytes/1024**2))
print("On-disk size (bytes):", disk_bytes, " (~{:.2f} MB)".format(disk_bytes/1024**2))
print("Meta:", out['meta'])

# Quick integrity check: dequantize a few large layers and print norm diffs vs original canon tensor
def dequantize_entry(info):
    qi = info['q'].to(torch.float32)
    sc = info['scale'].to(torch.float32)
    return (qi * sc).to(torch.float32)

print("\nSample dequant checks for a few largest quantized params:")
top_quant = sorted(quant_store.items(), key=lambda kv: -kv[1]['q'].numel())[:8]
for name, info in top_quant:
    deq = dequantize_entry(info)
    orig_t = canon_to_tensor[name]
    max_abs = (orig_t - deq).abs().max().item()
    rmse = ((orig_t - deq)**2).mean().sqrt().item()
    print(f" {name}: elems={info['q'].numel()}, bits={info['bits']}, dtype={info['dtype']}, max_abs={max_abs:.6e}, rmse={rmse:.6e}")

print("\nDone. If disk size still large, inspect fp32_fallback list above and adjust exclude_prefixes or bits_assignment.")


Loaded state_dict with 840 entries.
Canonical params found: 709
Prepared quant_store entries: 209
Prepared fp32_fallback entries: 500
Bytes: quant ints+scales: 18903904 fallback fp32: 1127104
Saved to: /content/drive/My Drive/Deep Fake Dataset/trained_cnn/quantized_all8_mapped.pth
Original (canon) FP32 total bytes (approx): 46802848  (~44.63 MB)
Quant stored ints+scales bytes (sum): 18903904  (~18460.84 KB)
Fallback FP32 bytes stored: 1127104  (~1.07 MB)
On-disk size (bytes): 20325211  (~19.38 MB)
Meta: {'num_quantized': 209, 'num_fallback_fp32': 500, 'total_quant_bytes': 18903904, 'total_fallback_bytes': 1127104}

Sample dequant checks for a few largest quantized params:
 backbone.features.7.1.block.0.0.weight: elems=884736, bits=4, dtype=int8, max_abs=6.609946e-02, rmse=1.201124e-02
 backbone.features.7.1.block.3.0.weight: elems=884736, bits=16, dtype=int16, max_abs=1.507252e-05, rmse=3.462930e-06
 dense_512.0.weight: elems=786432, bits=16, dtype=int16, max_abs=6.914139e-06, rmse=1.8

In [ ]:
import joblib, os, torch, numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

def dequantize_state_dict_from_disk(quant_path: str, device: str = "cpu"):
    """
    Loads a quantized checkpoint saved by save_quantized_checkpoint()
    or the mapped version, and reconstructs a full FP32 state_dict.

    Args:
        quant_path: path to quantized .pth file
        device: target device for returned tensors ('cpu' or 'cuda')

    Returns:
        OrderedDict mapping parameter names -> FP32 torch.Tensor
        This can be loaded into your model using model.load_state_dict(...)
    """

    import torch
    from collections import OrderedDict

    qfile = torch.load(quant_path, map_location="cpu")
    out_state = OrderedDict()

    # 1. Load fallback FP32 params
    fp32_fallback = qfile.get("fp32_fallback", {})
    for name, tensor in fp32_fallback.items():
        out_state[name] = tensor.to(torch.float32).to(device)

    # 2. Dequantize quantized params
    quant_map = qfile.get("quant", {})
    for name, info in quant_map.items():
        q_int = info["q"].to(torch.float32)           # integer tensor → float
        scale = info["scale"].to(torch.float32)       # per-channel / per-tensor scale
        w_fp32 = (q_int * scale).to(torch.float32)    # dequantized weight
        out_state[name] = w_fp32.to(device)

    return out_state


OUT_DIR = "/content/drive/My Drive/Deep Fake Dataset/trained_cnn/ensemble_classifiers"
os.makedirs(OUT_DIR, exist_ok=True)

# 1) Load quantized checkpoint and reconstruct fp32 state_dict (if using quant file)
quant_path = "/content/drive/My Drive/Deep Fake Dataset/trained_cnn/quantized_all8_mapped.pth"
# Use your dequantize helper to reconstruct a state_dict (function from earlier cell)
state_dict_fp32 = dequantize_state_dict_from_disk(quant_path, device='cpu')

# 2) Load into model
device = torch.device("cpu")
model = DeepFakeHybridModel().to(device)
model.load_state_dict(state_dict_fp32)
model.eval()

# 3) Extract features (this iterates your images; runs on CPU if device='cpu')
X_train, y_train = extract_features_from_disk('train', model)
X_val, y_val = extract_features_from_disk('val', model)

print("Feature shapes:", X_train.shape, X_val.shape)

# 4) Train classifiers
clfs = {
    'SVM': SVC(probability=True, kernel='rbf', random_state=42),
    'DT': DecisionTreeClassifier(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'NB': GaussianNB()
}

for name, clf in clfs.items():
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    acc = accuracy_score(y_val, preds)
    print(f"{name} val acc: {acc*100:.2f}%")
    # save classifier
    joblib.dump(clf, os.path.join(OUT_DIR, f"clf_{name}.joblib"))
    print("Saved", name)

# also save meta (order)
joblib.dump({"names": list(clfs.keys())}, os.path.join(OUT_DIR, "clf_meta.joblib"))
print("All classifiers saved in", OUT_DIR)
